## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head(10)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,10556855,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION...,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,10558440,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRI...,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,10566033,ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,10570430,ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,10571689,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [3]:
# Find unique values for SPECIAL_CONSIDERATIONS column
application_df["SPECIAL_CONSIDERATIONS"].unique()


array(['N', 'Y'], dtype=object)

In [4]:
# Find the counts of each unique value for the SPECIAL_CONSIDERATIONS column.
application_df["SPECIAL_CONSIDERATIONS"].value_counts()

# Because the unique values are either "N" or "Y", and we have so few Y values, we can add this column to our list of non-beneficial columns and drop it from the DataFrame.

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME", "SPECIAL_CONSIDERATIONS"], inplace=True)

In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# Using larger amount of bins to reduce the number of unique values
application_types_to_replace_1 = list(application_counts[(application_counts > 10) & (application_counts < 500)].index)

# Replace in dataframe
for app in application_types_to_replace_1:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other(250-500)")

# Second Binning
application_types_to_replace_2 = list(application_counts[application_counts < 10].index)

# Replace in dataframe
for app in application_types_to_replace_2:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other(<250)")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3                27037
T4                 1542
T6                 1216
T5                 1173
T19                1065
T8                  737
T7                  725
T10                 528
Other(250-500)      265
Other(<250)          11
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 750].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other(250-500),APPLICATION_TYPE_Other(<250),APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [13]:
# Split our preprocessed data into our features and target arrays
x = application_df.drop(columns=["IS_SUCCESSFUL"]).values
y = application_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Create a new model with further optimizations

In [15]:
# Import our dependencies
# Will be using keras_tuner to automatically tune the hyperparameters of our model
# This is a much more effective way of optimizing the model than manually adjusting parameters
import keras_tuner as kt

In [19]:
# Use keras tuner to find the best learning rate and parameters for the model

# Define the model
def create_nn_model(hp):
    # Create a sequential model
    nn = tf.keras.models.Sequential()

    # Add our first hidden layer where the input dimensions are the 45 columns of our dataset
    # Make Keras Tuner find the optimal number of nodes in this layer
    # Make Keras Tuner find the optimal activation function to use in this layer
    nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
                                        min_value=1,
                                        max_value=80,
                                        step=2), input_dim=43, activation=hp.Choice('first_activation', ['relu', 'tanh', 'sigmoid', 'softmax'])))
    
    # Make Keras Tuner find the optimal number of hidden layers to use
    # Make Keras Tuner find the optimal number of nodes to use in each hidden layer
    # Make Keras Tuner find the optimal activation function to use in each hidden layer
    for i in range(hp.Int('num_layers', 1, 10)):
        nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=1,
                                            max_value=80,
                                            step=2),
                               activation=hp.Choice('activation_' + str(i), ['relu', 'tanh', 'sigmoid', 'softmax'])))
        
    # Add our output layer
    # Make Keras Tuner find the optimal activation function to use in the output layer
    nn.add(tf.keras.layers.Dense(units=1, activation=hp.Choice('output_activation', ['relu', 'tanh', 'sigmoid', 'softmax'])))

    # Compile our model
    # Make Keras Tuner find the optimal learning rate to use
    nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), metrics=["accuracy"])

    # Return our model
    return nn
        


In [20]:
# Create a Keras Tuner Hyperband tuner
tuner = kt.Hyperband(create_nn_model,
                     objective='val_accuracy',
                     max_epochs=80,
                     factor=3,
                     directory='Testing_Paramater_Directory',
                     project_name='AlphabetSoupCharity_Optimization')

# Use the tuner to find the optimal hyperparameters
tuner.search(x_train_scaled,y_train,epochs=80,validation_data=(x_test_scaled,y_test))

Trial 84 Complete [00h 01m 47s]
val_accuracy: 0.5331778526306152

Best val_accuracy So Far: 0.7294460535049438
Total elapsed time: 00h 33m 48s


In [21]:
# Get the optimal hyperparameters from the tuner
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
best_hyperparameters.values

{'first_units': 39,
 'first_activation': 'tanh',
 'num_layers': 7,
 'units_0': 77,
 'activation_0': 'relu',
 'output_activation': 'relu',
 'learning_rate': 0.001,
 'units_1': 37,
 'activation_1': 'softmax',
 'units_2': 59,
 'activation_2': 'tanh',
 'units_3': 13,
 'activation_3': 'tanh',
 'units_4': 75,
 'activation_4': 'softmax',
 'units_5': 55,
 'activation_5': 'relu',
 'units_6': 11,
 'activation_6': 'relu',
 'units_7': 23,
 'activation_7': 'tanh',
 'units_8': 1,
 'activation_8': 'softmax',
 'tuner/epochs': 27,
 'tuner/initial_epoch': 9,
 'tuner/bracket': 3,
 'tuner/round': 2,
 'tuner/trial_id': '0037',
 'units_9': 13,
 'activation_9': 'softmax'}

In [22]:
# Create a new model with the optimal hyperparameters
optimized_model = tuner.get_best_models(1)[0]

# Evaluate the model using the test data
optimized_model_loss, optimized_model_accuracy = optimized_model.evaluate(x_test_scaled,y_test,verbose=2)

# Print the model loss and accuracy results
print(f"Loss: {optimized_model_loss}, Accuracy: {optimized_model_accuracy}")

268/268 - 1s - loss: 0.5503 - accuracy: 0.7294 - 502ms/epoch - 2ms/step
Loss: 0.5502783060073853, Accuracy: 0.7294460535049438


In [24]:
# Compile the model
optimized_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = optimized_model.fit(x_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5479 - accuracy: 0.7345
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7366
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7365
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7357
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5446 - accuracy: 0.7363
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5437 - accuracy: 0.7373
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5431 - accuracy: 0.7378
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5431 - accuracy: 0.7374
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accuracy: 0.7365
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5418 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = optimized_model.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5586 - accuracy: 0.7231 - 262ms/epoch - 978us/step
Loss: 0.5586250424385071, Accuracy: 0.7231487035751343


In [31]:
optimized_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 39)                1716      
                                                                 
 dense_1 (Dense)             (None, 77)                3080      
                                                                 
 dense_2 (Dense)             (None, 37)                2886      
                                                                 
 dense_3 (Dense)             (None, 59)                2242      
                                                                 
 dense_4 (Dense)             (None, 13)                780       
                                                                 
 dense_5 (Dense)             (None, 75)                1050      
                                                                 
 dense_6 (Dense)             (None, 55)                4

In [32]:
# Export our model to HDF5 file
optimized_model.save("AlphabetSoupCharity_Optimized.h5")

b:\CONDA\install\envs\amd_dev_2\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
